![image](https://www.ewatercycle.org/assets/logo.png)

In [ ]:
from pathlib import Path
import numpy as np
from ewatercycle import CFG
from ewatercycle.models import Lisflood
from ewatercycle.models.lisflood import LisfloodParameterSet

## Run lisflood class

The ewatercycle configuration file is typcially stored in `~/ewatercycle/ewatercycle.yaml` or `/etc/ewatercycle.yaml` and can be imported with `from ewatercycle import CFG`. Relevant settings for lisflood are:

```Yaml
container_engine: docker # or singularity
lisflood.singularity_images: 'ewatercycle-lisflood-grpc4bmi.sif'
lisflood.docker_images: 'ewatercycle/lisflood-grpc4bmi:latest'
output_dir: './' # the directory where the model work_dir will be created
```

In [ ]:
CFG.load_from_file('~/ewatercycle.yaml')

In [ ]:
parameterset = LisfloodParameterSet(
    root=Path('/projects/0/wtrcycle/comparison/lisflood_input/Lisflood01degree_masked'),
    mask=Path('/projects/0/wtrcycle/comparison/recipes_auxiliary_datasets/LISFLOOD/model_mask.nc'),
    config_template=Path('/projects/0/wtrcycle/comparison/lisflood_input/settings_templates/settings_lisflood.xml'),
    lisvap_config_template=Path('/projects/0/wtrcycle/comparison/lisflood_input/settings_templates/settings_lisvap.xml'),
)

In [ ]:
# create lisflood instance
model = Lisflood()

In [ ]:
# setup model
# lisflood_ERA-Interim_*_1990_1990.nc data can be copied 
# from /projects/0/wtrcycle/comparison/forcing/lisflood
forcing_dir = Path('forcing_data')
config_file, config_dir = model.setup(forcing_dir, parameterset)

In [ ]:
print(config_file)
print(config_dir)

In [ ]:
# get metadata
print(model.output_var_names)

In [ ]:
# run lisvap
lisvap_results = model.run_lisvap(forcing_dir)

In [ ]:
print(lisvap_results)

In [ ]:
# initialize
model.initialize(str(config_file))

In [ ]:
# run model with bmi
model.update()

In [ ]:
# get value
discharge = model.get_value('Discharge')

In [ ]:
np.count_nonzero(~np.isnan(discharge))

In [ ]:
discharge = model.get_value_as_xarray('Discharge')

In [ ]:
discharge.plot(robust=True, cmap='GnBu')

In [ ]:
# stop the model
del model.bmi